# Parametrization of Merger Trees

In [ ]:
# Import python modules
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import copy
import imp
import os
import glob
import sys
import time

In [ ]:
# Define path to the codes
cagadir = '/Users/benoitcote/Desktop/Carleen/Code_Carleen/caga/caga'
sygmadir = '/Users/benoitcote/Desktop/Carleen/Code_Carleen/NuPyCEE'
jinapydir = '/Users/benoitcote/Desktop/Carleen/Code_Carleen/JINAPyCEE'

# Set the environments to let Python knows where are the codes
os.environ['CAGADIR'] = cagadir
os.environ['SYGMADIR'] = sygmadir
os.environ['JINAPYDIR'] = jinapydir

# Import the codes
caga  = imp.load_source('caga', cagadir+'/caga.py')
calc  = imp.load_source('calc', cagadir+'/calc.py')
plot  = imp.load_source('plot', cagadir+'/plot.py')
sygma = imp.load_source('sygma', sygmadir+'/sygma.py')
omega = imp.load_source('omega', sygmadir+'/omega.py')
gamma = imp.load_source('gamma', jinapydir+'/gamma.py')
omega_plus = imp.load_source('omega_plus', jinapydir+'/omega_plus.py')

In [ ]:
# Get the file name for the host tree (Milky-Way-like halo)
hostID = 686
hostfname = cagadir+"/../notebooks/H1725272_LX11/rsid{}.npy".format(hostID)

# Get the file names for the satellite sub-trees
# Below, the host tree will be removed from that list
subfnames = glob.glob(cagadir+"/../notebooks/H1725272_LX11/*")

# Convert \ into /, (happens sometime with Windows machines)
# Here should be the script we made ..

# Remove the host tree file name from the sub-tree file names
subfnames.remove(hostfname)
print(len(subfnames),'sub-trees found')

In [ ]:
# Load the GAMMA input arrays for each tree
host_tree = caga.gamma_tree.load(hostfname)
sub_trees = [caga.gamma_tree.load(subfname) for subfname in subfnames]

In [ ]:
# Get the indexes for the branch that are merging together
def get_merging_branch_index(the_tree, desc_ID):
    i_z_br_merging = []
    for i_z_gmbi in range(len(the_tree.br_m_halo)):
        for i_br_gmbi in range(len(the_tree.br_m_halo[i_z_gmbi])):
            if the_tree.br_ID_merge[i_z_gmbi][i_br_gmbi] == desc_ID:
                i_z_br_merging.append([i_z_gmbi,i_br_gmbi])
    return i_z_br_merging

# Find the initial mass of the branch that ends up being the trunk
def get_initial_trunk_index(the_tree):
    for i_z_giti in range(len(the_tree.br_halo_ID)):
        for i_br_giti in range(len(the_tree.br_halo_ID[i_z_giti])):
            if the_tree.br_halo_ID[i_z_giti][i_br_giti][-1] == the_tree.tree_trunk_ID:
                return i_z_giti, i_br_giti
    print('Error - trunk not found.')
    return -1, -1

In [ ]:
# Get the evolution of the mass of the main branch of a given tree
def get_mass_main_branch(the_tree):

    # Define the number of redshifts and initialize the list of mass for the main branch
    nb_z = len(the_tree.redshifts)
    m_main_branch = np.zeros(nb_z)

    # Initialize the branch index (begining of the branch that becomes the trunk)
    the_i_z, the_i_br = get_initial_trunk_index(the_tree)
    the_m = the_tree.br_m_halo[the_i_z][the_i_br]
    for i_z in range(len(the_m)):
        m_main_branch[the_i_z+i_z] = the_m[i_z]

    # While we have not yet covered all redshifts ..
    while the_i_z >= 0:

        # Get all branches that merged to form the current branch
        i_z_br_merging = get_merging_branch_index(the_tree, the_tree.br_halo_ID[the_i_z][the_i_br][0])

        # Quit if we reached a leaf
        len_i_z_br_merging = len(i_z_br_merging)
        if len_i_z_br_merging == 0:
            break

        # Find the branch index that has the most massive end point
        m_max, i_z_br_max = -1, 0
        for i_z_br in range(len_i_z_br_merging):
            test_m = the_tree.br_m_halo[i_z_br_merging[i_z_br][0]][i_z_br_merging[i_z_br][1]][-1]
            if test_m > float(m_max):
                m_max = ('%.3e'%copy.deepcopy(test_m))
                i_z_br_max = copy.deepcopy(i_z_br)

        # Go to the previous step
        # Begining of the most massive building-block branch
        the_i_z = copy.deepcopy(i_z_br_merging[i_z_br_max][0])
        the_i_br = copy.deepcopy(i_z_br_merging[i_z_br_max][1])

        # Collect the masses
        the_m = the_tree.br_m_halo[the_i_z][the_i_br]
        for i_z in range(len(the_m)):
            m_main_branch[the_i_z+i_z] = the_m[i_z]
            
    # Return the mass history
    return m_main_branch

In [ ]:
# Extract the mass history of the main branch of a given tree
the_tree = host_tree
#the_tree = sub_trees[5]
m_main_branch = get_mass_main_branch(the_tree)

In [ ]:
# Plot the full mass history along with the extracted main branch
%matplotlib nbagg
the_tree.plot_mass_history()
plt.plot(the_tree.redshifts, m_main_branch, color='c', linewidth=8, alpha=0.4, zorder=-1, label='Main branch')
plt.legend(frameon=False, fontsize=13)
plt.savefig('a.pdf', dpi=150, bbox_inches='tight')